# Optical Flow Random Shaking Iterative Volume Denoising by Slices

In [ ]:
local_debug = True

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    from google.colab import drive
    drive.mount('/content/drive')
    !cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    !cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
#import denoising.image.OF_random as denoising

In [ ]:
from motion_estimation._2D.farneback_OpenCV import Estimator_in_CPU as Estimator
from motion_estimation._2D.project import project

In [ ]:
#import opticalflow3D
#import warnings
#from numba.core.errors import NumbaPerformanceWarning
#import numpy as np

#warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
import skimage.io

In [ ]:
import numpy as np

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
args = Args("/home/vruiz/cryoCARE/tomo2_L1G1_bin4.rec", "denoised")

In [ ]:
import time

In [ ]:
import cv2

In [ ]:
import threading

In [ ]:
import mrcfile
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1I2uIfM00ZNeMjYy4OeZ4hSO-bxpE3oZb"
    gdown https://drive.google.com/uc?id=$FILEID
fi
set +x
# https://drive.google.com/file/d/1I2uIfM00ZNeMjYy4OeZ4hSO-bxpE3oZb/view?usp=sharing

noisy = opticalflow3D.helpers.load_image(args.input)

In [ ]:
np.min(noisy)

In [ ]:
np.max(noisy)

In [ ]:
#noisy = (255*(noisy.astype(np.float32) - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)
noisy = (255*(noisy.astype(np.float32) - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.float32)

In [ ]:
np.min(noisy)

In [ ]:
np.max(noisy)

In [ ]:
#noisy = noisy[230:250, 450:760, 210:510]
noisy = noisy[180:220, 400:800, 200:600]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 16))
axs.imshow(noisy[:, ::-1, :][10], cmap="gray")
axs.set_title(f"Noisy")
fig.tight_layout()
plt.show()

In [ ]:
noisy.shape

In [ ]:
PYRAMID_LEVELS = 3
WINDOW_SIDE = 5
NUM_ITERATIONS = 5
N_POLY = 5
#POLY_SIGMA = 1.2
PYR_SCALE = 0.5

#class Estimator_in_CPU(logging.Logger):
class Estimator(logging.Logger):
    
    def __init__(
        self,
        pyr_levels=PYRAMID_LEVELS,
        pyr_scale=PYR_SCALE,
        win_side=WINDOW_SIDE,      # Applicability window side
        num_iters=NUM_ITERATIONS,  # Number of iterations at each pyramid level
        #sigma_poly=POLY_SIGMA,     
        sigma_N=N_POLY,            
        logging_level=logging.INFO
    ):
        self.logger = logging.getLogger(__name__)
        self.pyr_levels = pyr_levels
        self.pyr_scale = pyr_scale
        self.win_side = win_side
        self.num_iters = num_iters
        #self.sigma_poly = sigma_poly
        self.N_poly = N_POLY
        self.flags = 0
        logger.setLevel(logging_level)
        
        for attr, value in vars(self).items():
            logger.debug(f"{attr}: {value}")

        if logger.getEffectiveLevel() <= logging.INFO:
            self.running_time = 0
            self.total_running_time = 0

    def get_times(self):
        return self.running_time

    def use_previuos_flow():
        self.flags |= cv2.OPTFLOW_USE_INITIAL_FLOW

    def use_gaussian_applicability():
        self.flags |= cv2.OPTFLOW_FARNEBACK_GAUSSIAN

    def pyramid_get_flow(
        self,
        target,
        reference,
        flow=None
    ):

        if self.logger.getEffectiveLevel() <= logging.INFO:
            time_0 = time.perf_counter()

        sigma_poly = (self.N_poly - 1)/4 # Standard deviation of the Gaussian basis used in the polynomial expansion
        self.logger.debug(f"sigma_poly={sigma_poly}")
        #print(target.shape, reference.shape, flow, target.dtype, reference.dtype)
        flow = cv2.calcOpticalFlowFarneback(
            prev=target,
            next=reference,
            flow=flow,
            pyr_scale=self.pyr_scale,
            levels=self.pyr_levels,
            winsize=self.win_side,
            iterations=self.num_iters,
            poly_n=self.N_poly,
            poly_sigma=sigma_poly,
            flags=self.flags)

        if self.logger.getEffectiveLevel() <= logging.INFO:
            time_1 = time.perf_counter()
            last_running_time = time_1 - time_0
            self.total_running_time += last_running_time

        self.logger.debug(f"avg_OF={np.average(np.abs(flow)):4.2f}")

        return flow


In [ ]:
self_max = 0
self_min = 0
#self_avg = 0

def shake_vector(x, std_dev=1.0):
  y = np.arange(len(x))
  displacements = np.random.normal(0, std_dev, len(x))
  #print(f"{np.min(displacements):.2f} {np.average(np.abs(displacements)):.2f} {np.max(displacements):.2f}", end=' ')
  global self_max, self_min, self_avg
  _min = np.min(displacements)
  #_avg = np.average(np.abs(displacements))
  _max = np.max(displacements)
  if _min < self_min:
      self_min = _min
  if _max > self_max:
      self_max = _max
  #if _avg < self_avg:
  #    self_avg = _avg
  return np.stack((y + displacements, x), axis=1)

def shake_slice(slice, mean=0.0, std_dev=1.0):
    global self_max, self_min, self_avg
    print(slice.shape)
    print(std_dev)
    shaked_slice = np.empty_like(slice)

    # Randomization in Y
    values = np.arange(slice.shape[0]).astype(np.int16)
    for x in range(slice.shape[1]):
        #print(x, end=' ')
        pairs = shake_vector(values, std_dev).astype(np.int16)
        pairs = pairs[pairs[:, 0].argsort()]
        shaked_slice[values, x] = slice[pairs[:, 1], x]
    slice = np.copy(shaked_slice)

    # Randomization in X
    values = np.arange(slice.shape[1]).astype(np.int16)
    for y in range(slice.shape[0]):
        #print(y, end=' ')
        pairs = shake_vector(values, std_dev).astype(np.int16)
        pairs = pairs[pairs[:, 0].argsort()]
        shaked_slice[y, values] = slice[y, pairs[:, 1]]

    print(f"\nmin={self_min} max={self_max}")
    return shaked_slice

def project_slice_A_to_B(farneback, A, B):
    #flow = self.get_flow_to_project_A_to_B(A, B)
    flow = farneback.pyramid_get_flow(target=B, reference=A, flow=None)
    print(f"{np.min(flow)} {np.average(np.abs(flow))} {np.max(flow)}")
    projection = project(logger, image=A, flow=flow)
    return projection

def filter_slice(farneback, noisy_slice, denoised_slice, RS_sigma, RS_mean):
    global self_max, self_min
    self_max = 0
    self_min = 0
    shaked_noisy_slice = shake_slice(noisy_slice, mean=RS_mean, std_dev=RS_sigma)
    shaked_and_compensated_noisy_slice = project_slice_A_to_B(farneback, A=denoised_slice, B=shaked_noisy_slice)
    return shaked_and_compensated_noisy_slice

def filter_vol(farneback, noisy_vol, N_iters=25, RS_sigma=2.0, RS_mean=0.0):
    acc_vol = np.zeros_like(noisy_vol, dtype=np.float32)
    acc_vol[...] = noisy_vol
    for i in range(N_iters):
        print(f"iter={i}")
        denoised_vol = acc_vol/(i+1)
        for z in range(noisy_vol.shape[0]):
            print(f"z={z}", end=' ')
            acc_vol[z, :, :] += filter_slice(farneback, noisy_vol[z, :, :], denoised_vol[z, :, :], RS_sigma, RS_mean)
        for y in range(noisy_vol.shape[1]):
            print(f"y={y}", end=' ')
            acc_vol[:, y, :] += filter_slice(farneback, noisy_vol[:, y, :], denoised_vol[:, y, :], RS_sigma, RS_mean)
        for x in range(noisy_vol.shape[2]):
            print(f"x={x}", end=' ')
            acc_vol[:, :, x] += filter_slice(farneback, noisy_vol[:, :, x], denoised_vol[:, :, x], RS_sigma, RS_mean)
    denoised_vol = acc_vol/(N_iters + 1)
    return denoised_vol

In [ ]:
#from motion_estimation._2D.farneback_OpenCV import Estimator_in_CPU as OF_Estimator

PYRAMID_LEVELS = 3
WINDOW_SIDE = 5
ITERATIONS = 5
N_POLY = 5
#POLY_SIGMA = 1.2
PYR_SCALE = 0.5

class OF_Estimation(logging.Logger):
    
    def __init__(
        self,      
        logging_level=logging.INFO
    ):
        self.logger = logging.getLogger(__name__)
        #self.flags = 0
        self.logger.setLevel(logging_level)
        
        for attr, value in vars(self).items():
            self.logger.debug(f"{attr}: {value}")

    def pyramid_get_flow(
        self,
        target,
        reference,
        flow=None,
        pyramid_levels=PYRAMID_LEVELS, # Number of pyramid layers
        window_side=WINDOW_SIDE,       # Applicability window side
        iterations=ITERATIONS,         # Number of iterations at each pyramid level
        N_poly=N_POLY,                 # Standard deviation of the Gaussian basis used in the polynomial expansion
        flags=0                        # cv2.OPTFLOW_USE_INITIAL_FLOW | cv2.OPTFLOW_FARNEBACK_GAUSSIAN
    ):

        sigma_poly = (N_poly - 1)/4 # Standard deviation of the Gaussian basis used in the polynomial expansion
        self.logger.debug(f"sigma_poly={sigma_poly}")
        #print(target.shape, reference.shape, flow, target.dtype, reference.dtype)
        flow = cv2.calcOpticalFlowFarneback(
            prev=target,
            next=reference,
            flow=flow,
            pyr_scale=0.5,
            levels=pyramid_levels,
            winsize=window_side,
            iterations=iterations,
            poly_n=N_poly,
            poly_sigma=sigma_poly,
            flags=flags)

        return flow
        
class Slice_Projection(logging.Logger):
    
    def __init__(
        self,
        logging_level=logging.INFO
    ):
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging_level)

    def remap(self, slice, flow, interpolation_mode=cv2.INTER_LINEAR, extension_mode=cv2.BORDER_REPLICATE):
        height, width = flow.shape[:2]
        map_x = np.tile(np.arange(width), (height, 1))
        map_y = np.swapaxes(np.tile(np.arange(height), (width, 1)), 0, 1)
        map_xy = (flow + np.dstack((map_x, map_y))).astype('float32')
        projection = cv2.remap(
            slice,
            map_xy,
            None,
            interpolation=interpolation_mode,
            borderMode=extension_mode)
        return projection

    def add_coordinates(flow, target):
        return flow + np.moveaxis(np.indices(target.shape), 0, -1)
 
class Random_Shaking_Denoising(OF_Estimation, Slice_Projection):
    def __init__(
        self,
        logging_level=logging.INFO
    ):
        OF_Estimation.__init__(self, logging_level)
        Slice_Projection.__init__(self, logging_level)
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging_level)
        if self.logger.getEffectiveLevel() <= logging.INFO:
            self.max = 0
            self.min = 0
        print(f"{'iter':>5s}", end='')
        print(f"{'min_shaking':>15s}", end='')
        print(f"{'max_shaking':>15s}", end='')
        print(f"{'min_flow':>15s}", end='')
        print(f"{'avg_abs_flow':>15s}", end='')
        print(f"{'max_flow':>15s}", end='')
        print(f"{'time':>15s}", end='')
        print()

        self.stop_event = threading.Event()
        self.logger_daemon = threading.Thread(target=self.show_log)
        self.logger_daemon.daemon = True
        self.time_0 = time.perf_counter()
        self.logger_daemon.start()

    def show_log(self):
        #while not self.stop_event.is_set():
        while self.stop_event.wait():
            time_1 = time.perf_counter()
            running_time = time_1 - self.time_0
            print(f"{self.iter:>5d}", end='')
            print(f"{np.min(self.displacements):>15.2f}", end='')
            print(f"{np.max(self.displacements):>15.2f}", end='')
            print(f"{np.min(self.flow):>15.2f}", end='')
            print(f"{np.average(np.abs(self.flow)):>15.2f}", end='')
            print(f"{np.max(self.flow):>15.2f}", end='')
            print(f"{running_time:>15.2f}", end='')
            print()
            self.stop_event.clear()
            self.time_0 = time.perf_counter()

    def shake_vector(self, x, mean=0.0, std_dev=1.0):
        y = np.arange(len(x))
        self.displacements = np.random.normal(mean, std_dev, len(x))
        return np.stack((y + self.displacements, x), axis=1)

    def shake_slice(self, slc, mean=0.0, std_dev=1.0):
        shaked_slice = np.empty_like(slc)
    
        # Shaking in Y
        values = np.arange(slc.shape[0]).astype(np.int16)
        for x in range(slc.shape[1]):
            pairs = self.shake_vector(x=values, mean=mean, std_dev=std_dev).astype(np.int16)
            pairs = pairs[pairs[:, 0].argsort()]
            shaked_slice[values, x] = slc[pairs[:, 1], x]
        slc = shaked_slice
    
        # Shaking in X
        values = np.arange(slc.shape[1]).astype(np.int16)
        for y in range(slc.shape[0]):
            pairs = self.shake_vector(x=values, mean=mean, std_dev=std_dev).astype(np.int16)
            pairs = pairs[pairs[:, 0].argsort()]
            shaked_slice[y, values] = slc[y, pairs[:, 1]]
    
        return shaked_slice

    def project_slice_reference_to_target(self, reference, target, pyramid_levels, window_side, iterations, N_poly, interpolation_mode, extension_mode):
        self.flow = self.pyramid_get_flow(
            target=target,
            reference=reference,
            flow=None,
            pyramid_levels=pyramid_levels,
            window_side=window_side,
            iterations=iterations,
            N_poly=N_poly)
        projection = self.remap(reference, self.flow, interpolation_mode, extension_mode)
        return projection

    def filter_slice(self, noisy_slice, denoised_slice, mean, std_dev, pyramid_levels, window_side, iterations, N_poly, interpolation_mode, extension_mode):
        shaked_noisy_slice = self.shake_slice(slc=noisy_slice, mean=mean, std_dev=std_dev)
        
        shaked_and_compensated_noisy_slice = self.project_slice_reference_to_target(
            reference=denoised_slice,
            target=shaked_noisy_slice,
            pyramid_levels=pyramid_levels,
            window_side=window_side,
            iterations=iterations,
            N_poly=N_poly,
            interpolation_mode=interpolation_mode,
            extension_mode=extension_mode)
        return shaked_and_compensated_noisy_slice

    def filter_vol(
        self,
        noisy_vol,
        N_iters=25,
        mean=0.0,
        std_dev=1.0,
        pyramid_levels=3,
        window_side=5,
        iterations=2,
        N_poly=5,
        interpolation_mode=cv2.INTER_LINEAR,
        extension_mode=cv2.BORDER_REPLICATE
    ):
        acc_vol = np.zeros_like(noisy_vol, dtype=np.float32)
        acc_vol[...] = noisy_vol
        for i in range(N_iters):
            self.iter = i
            denoised_vol = acc_vol/(i+1)

            for z in range(noisy_vol.shape[0]):
                acc_vol[z, :, :] += self.filter_slice(
                    noisy_slice=noisy_vol[z, :, :],
                    denoised_slice=denoised_vol[z, :, :],
                    mean=mean,
                    std_dev=std_dev,
                    pyramid_levels=pyramid_levels,
                    window_side=window_side,
                    iterations=iterations,
                    N_poly=N_poly,
                    interpolation_mode=interpolation_mode,
                    extension_mode=extension_mode)
            self.stop_event.set()

            for y in range(noisy_vol.shape[1]):
                acc_vol[:, y, :] += self.filter_slice(
                    noisy_slice=noisy_vol[:, y, :],
                    denoised_slice=denoised_vol[:, y, :],
                    mean=mean,
                    std_dev=std_dev,
                    pyramid_levels=pyramid_levels,
                    window_side=window_side,
                    iterations=iterations,
                    N_poly=N_poly,
                    interpolation_mode=interpolation_mode,
                    extension_mode=extension_mode)
            self.stop_event.set()

            for x in range(noisy_vol.shape[2]):
                acc_vol[:, :, x] += self.filter_slice(
                    noisy_slice=noisy_vol[:, :, x],
                    denoised_slice=denoised_vol[:, :, x],
                    mean=mean,
                    std_dev=std_dev,
                    pyramid_levels=pyramid_levels,
                    window_side=window_side,
                    iterations=iterations,
                    N_poly=N_poly,
                    interpolation_mode=interpolation_mode,
                    extension_mode=extension_mode)
            self.stop_event.set()

        denoised_vol = acc_vol/(N_iters + 1)
        return denoised_vol

In [ ]:
RS_std_dev = 1.0
N_iters = 25

In [ ]:
denoiser = Random_Shaking_Denoising()
denoised = denoiser.filter_volume(noisy, std_dev=RS_std_dev, N_iters=N_iters)

In [ ]:
#denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=N_iters)
farneback = Estimator(pyr_levels=3, pyr_scale=0.5, win_side=5, num_iters=2, sigma_N=5, logging_level=logging.INFO)
denoised = filter_vol(farneback, noisy, RS_sigma=RS_sigma, N_iters=N_iters)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(32, 64))
#axs[0].imshow(noisy[:, ::-1, :][210][400:800, 200:600], cmap="gray")
axs[0].imshow(noisy[:, ::-1, :][10], cmap="gray")
axs[0].set_title(f"Noisy")
#axs[1].imshow(denoised[:, ::-1, :][210][400:800, 200:600], cmap="gray")
axs[1].imshow(denoised[:, ::-1, :][10], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[10], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(cryo[80+10, 150:300, 150:300], cmap="gray")
axs[1].set_title(f"cryoCARE (DQI={information_theory.information.compute_quality_index(noisy[10], cryo[80+10, 150:300, 150:300])})")
fig.tight_layout()
plt.show()

fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[131][300:,300:], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[131][300:,300:], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[131][300:,300:], denoised[131][300:,300:])}")
fig.tight_layout()
plt.show()

fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy[137][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[137][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[137][::-1, :] - denoised[137][::-1, :], cmap="gray")

In [ ]:
print(f"{args.output}_{RS_std_dev}_{N_iters}.mrc")
with mrcfile.new(f"{args.output}_{RS_std_dev}_{N_iters}.mrc", overwrite=True) as mrc:
            mrc.set_data(denoised.astype(np.float32))
            mrc.data
skimage.io.imsave(f"{args.output}_{RS_std_dev}_{N_iters}.tif", denoised, imagej=True)

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
import denoising.image.OF_random as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoiser(
    logger,
    pyramid_levels = 3,
    window_side = 15,
    N_poly = 5,
    num_iterations = 10
)

In [ ]:
denoised, _ = denoiser.filter(noisy[10], None, N_iters=300, RS_sigma=0.5)

In [ ]:
print(np.min(denoised), np.max(denoised))

In [ ]:
denoised = np.clip(denoised, a_min=0, a_max=255)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[10], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[10], denoised)})")
fig.tight_layout()
plt.show()

In [ ]:
input()

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=4,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
RS_sigma = 2.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")